In [3]:
# import driver from google colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
##Extract the zip file
# import zipfile

# zip_file_path = "/content/drive/MyDrive/Cap_v1/Food_101.zip"  # Replace with actual path
# extract_to_path = "/content/drive/MyDrive/Cap_v1/"  # Replace with desired extraction folder

# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_to_path)

print("Extraction completed successfully!")

Extraction completed successfully!


In [ ]:
!pip install numpy==1.25.2 pandas==2.0.3 seaborn==0.13.1 tensorflow==2.15.0 scikit-learn==1.2.2 matplotlib==3.7.1

ERROR: Operation cancelled by user


In [1]:
import os
import numpy as np                                                                               # Importing numpy for Matrix Operations
import pandas as pd                                                                              # Importing pandas to read CSV files
import matplotlib.pyplot as plt                                                                  # Importting matplotlib for Plotting and visualizing images
import math                                                                                      # Importing math module to perform mathematical operations
import cv2                                                                                       # Importing openCV for image processing
import seaborn as sns                                                                            # Importing seaborn to plot graphs


# Tensorflow modules
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator                              # Importing the ImageDataGenerator for data augmentation
from tensorflow.keras.models import Sequential                                                   # Importing the sequential module to define a sequential model
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D,BatchNormalization # Defining all the layers to build our CNN Model
from tensorflow.keras.optimizers import Adam,SGD                                                 # Importing the optimizers which can be used in our model
from sklearn import preprocessing                                                                # Importing the preprocessing module to preprocess the data
from sklearn.model_selection import train_test_split                                             # Importing train_test_split function to split the data into train and test
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Model
from keras.applications.vgg16 import VGG16                                               # Importing confusion_matrix to plot the confusion matrix

# Display images using OpenCV
from google.colab.patches import cv2_imshow

#Imports functions for evaluating the performance of machine learning models
from sklearn.metrics import confusion_matrix, f1_score,accuracy_score, recall_score, precision_score, classification_report
from sklearn.metrics import mean_squared_error as mse                                                 # Importing cv2_imshow from google.patches to display images

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [5]:
import os
import random
from PIL import Image
import json

# Define the path to the dataset
dataset_path = '/content/drive/MyDrive/Cap_v1/Food_101'  # Replace with the actual path if different

# Automatically list food classes from the directory
food_classes = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]

# Select two random classes (you can modify this to select more or specific ones)
selected_food_classes = random.sample(food_classes, 10)  # Selecting randomly to avoid hardcoding

print("Selected Food Classes:", selected_food_classes)

# Number of images per class for annotation
num_images_per_class = 50

# Paths for training and testing data (assuming a standard split)
train_path = os.path.join(dataset_path, '')
test_path = os.path.join(dataset_path, '')

Selected Food Classes: ['falafel', 'ravioli', 'donuts', 'ice_cream', 'chocolate_cake', 'onion_rings', 'pancakes', 'nachos', 'waffles', 'samosa']


In [6]:
train_image_paths = {}
test_image_paths = {}

for food_class in selected_food_classes:
    train_image_paths[food_class] = []
    test_image_paths[food_class] = []
    class_path = os.path.join(train_path, food_class)
    all_images = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
    selected_images = random.sample(all_images, min(num_images_per_class, len(all_images))) # Select up to 50 images

    # For this milestone, we'll consider these selected images for both 'training' and annotation
    for img_name in selected_images:
        train_image_paths[food_class].append(os.path.join(class_path, img_name))
        test_image_paths[food_class].append(os.path.join(class_path, img_name))
print(train_image_paths)
print(test_image_paths)
print("Training image paths for selected classes:")
for cls, paths in train_image_paths.items():
    print(f"{cls}: {len(paths)} images")

print("\nTesting image paths for selected classes:")
for cls, paths in test_image_paths.items():
    print(f"{cls}: {len(paths)} images")

{'falafel': ['/content/drive/MyDrive/Cap_v1/Food_101/falafel/33628.jpg', '/content/drive/MyDrive/Cap_v1/Food_101/falafel/2379286.jpg', '/content/drive/MyDrive/Cap_v1/Food_101/falafel/2573122.jpg', '/content/drive/MyDrive/Cap_v1/Food_101/falafel/2018817.jpg', '/content/drive/MyDrive/Cap_v1/Food_101/falafel/1530002.jpg', '/content/drive/MyDrive/Cap_v1/Food_101/falafel/1986946.jpg', '/content/drive/MyDrive/Cap_v1/Food_101/falafel/3145303.jpg', '/content/drive/MyDrive/Cap_v1/Food_101/falafel/2438806.jpg', '/content/drive/MyDrive/Cap_v1/Food_101/falafel/1865894.jpg', '/content/drive/MyDrive/Cap_v1/Food_101/falafel/2977898.jpg', '/content/drive/MyDrive/Cap_v1/Food_101/falafel/3083605.jpg', '/content/drive/MyDrive/Cap_v1/Food_101/falafel/157142.jpg', '/content/drive/MyDrive/Cap_v1/Food_101/falafel/1700313.jpg', '/content/drive/MyDrive/Cap_v1/Food_101/falafel/3682673.jpg', '/content/drive/MyDrive/Cap_v1/Food_101/falafel/1189019.jpg', '/content/drive/MyDrive/Cap_v1/Food_101/falafel/861737.jpg',

In [7]:
import matplotlib.pyplot as plt
import cv2
import os
import random # Import random if you want to display a random sample

# Assume 'train_image_paths' and 'test_image_paths' dictionaries are defined from previous steps
# They should look something like:
# train_image_paths = {
#     'apple_pie': ['path/to/train/apple_pie/img1.jpg', ...],
#     'pizza': ['path/to/train/pizza/imgA.jpg', ...]
# }
# test_image_paths = {
#     'apple_pie': ['path/to/test/apple_pie/imgX.jpg', ...],
#     'pizza': ['path/to/test/pizza/imgY.jpg', ...]
# }

def display_images_with_labels(image_data, title, num_rows=10, num_cols=5, figure_size=(10, 15)):
    """Displays images with their corresponding labels in a grid with a main title.

    Args:
        image_data: A list of tuples, where each tuple contains (image_path, label).
        title: The main title for the figure (e.g., "Training Images").
        num_rows: Number of rows in the display grid.
        num_cols: Number of columns in the display grid.
        figure_size: Size of the Matplotlib figure.
    """
    num_images = len(image_data)
    fig, axes = plt.subplots(num_rows, num_cols, figsize=figure_size)
    fig.suptitle(title, fontsize=16) # Set the main title for the figure
    axes = axes.flatten()  # Flatten the axes array for easy indexing

    # Turn off all axes by default, in case there are more subplots than images
    for ax in axes:
        ax.axis('off')

    for i, (img_path, label) in enumerate(image_data):
        if i >= len(axes):
            print(f"Warning: More images ({num_images}) than available subplots ({len(axes)}). Displaying only the first {len(axes)}.")
            break # Stop if we've used all allocated subplots

        try:
            img = cv2.imread(img_path)
            if img is not None:
                img_resized = cv2.resize(img, (169, 169)) # You can change (224, 224) to other values
                img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
                axes[i].imshow(img_rgb)
                axes[i].set_title(label)  # Set the title to the class label
                axes[i].axis('on') # Turn on the axis for the used subplot if it was off
            else:
                axes[i].text(0.5, 0.5, f"Error loading\n{os.path.basename(img_path)}", ha='center', va='center', color='red')
                axes[i].axis('off')
        except Exception as e:
            axes[i].text(0.5, 0.5, f"Error: {e}\n{os.path.basename(img_path)}", ha='center', va='center', color='red')
            axes[i].axis('off')


    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to make space for the suptitle
    plt.show()

# --- Prepare Data for Display ---

# Prepare training image data with labels
train_image_data_with_labels = []
for class_name, paths in train_image_paths.items():
    for path in paths:
        train_image_data_with_labels.append((path, class_name))

# Prepare testing image data with labels
test_image_data_with_labels = []
# Assuming a similar structure for test_image_paths
for class_name, paths in test_image_paths.items():
    for path in paths:
        test_image_data_with_labels.append((path, class_name))

# --- Display Images with Headers ---

# Display Training Images
print("--- Training Images ---")
# You can choose to display all or a random sample
# display_images_with_labels(train_image_data_with_labels, "Training Images", num_rows=len(train_image_paths), num_cols=5) # Display all
# Or display a random sample:
num_train_to_display = 50 # Number of random training images to display
random_train_data = random.sample(train_image_data_with_labels, min(num_train_to_display, len(train_image_data_with_labels)))
display_images_with_labels(random_train_data, "Training Images", num_rows=4, num_cols=5) # Adjust rows/cols as needed

# Display Testing Images
print("\n--- Testing Images ---") # Add a newline for separation
# You can choose to display all or a random sample
# display_images_with_labels(test_image_data_with_labels, "Testing Images", num_rows=len(test_image_paths), num_cols=5) # Display all
# Or display a random sample:
num_test_to_display = 50 # Number of random testing images to display
random_test_data = random.sample(test_image_data_with_labels, min(num_test_to_display, len(test_image_data_with_labels)))
display_images_with_labels(random_test_data, "Testing Images", num_rows=4, num_cols=5) # Adjust rows/cols as needed

Output hidden; open in https://colab.research.google.com to view.

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def build_basic_cnn(input_shape, num_classes):
    model = Sequential()

    # First Convolutional Block
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))

    # Second Convolutional Block
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    # Third Convolutional Block (Optional, but good for slightly more complexity)
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    # Flatten the 3D output to 1D for the Dense layers
    model.add(Flatten())

    # Dense (Fully Connected) Layers
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5)) # Dropout for regularization to prevent overfitting
    model.add(Dense(num_classes, activation='softmax')) # Output layer for classification

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [16]:
import os
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split # Used for splitting if needed, or just for clarity
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# --- 1. Define your data paths (IMPORTANT: REPLACE WITH YOUR ACTUAL PATHS) ---
# Assuming you've already run the data splitting logic from previous steps
# If not, you'll need to generate these dictionaries first.

# Example structure (replace with your actual data)
# train_image_paths = {
#     'apple_pie': ['path/to/train/apple_pie/img1.jpg', 'path/to/train/apple_pie/img2.jpg'],
#     'pizza': ['path/to/train/pizza/imgA.jpg', 'path/to/train/pizza/imgB.jpg'],
#     'sushi': ['path/to/train/sushi/imgX.jpg', 'path/to/train/sushi/imgY.jpg'],
# }
# test_image_paths = {
#     'apple_pie': ['path/to/test/apple_pie/img_test1.jpg'],
#     'pizza': ['path/to/test/pizza/img_testA.jpg'],
#     'sushi': ['path/to/test/sushi/img_testX.jpg'],
# }

# --- Dummy Data Setup (for quick testing if you don't have real data yet) ---
# You can uncomment and modify this section to create dummy files
# if you want to test the code without a full dataset.
# Make sure to replace 'path/to/your/food_dataset' with a real, accessible directory.

# base_data_dir = 'food_dataset_dummy' # Change this to a real path if needed
# selected_food_classes = ['pizza', 'sushi', 'burger']
# num_images_per_class_dummy = 20 # For train
# num_test_images_per_class_dummy = 5 # For test

# train_image_paths = {}
# test_image_paths = {}

# for food_class in selected_food_classes:
#     train_image_paths[food_class] = []
#     test_image_paths[food_class] = []
#     class_train_path = os.path.join(base_data_dir, 'train', food_class)
#     class_test_path = os.path.join(base_data_dir, 'test', food_class)
#     os.makedirs(class_train_path, exist_ok=True)
#     os.makedirs(class_test_path, exist_ok=True)

#     for i in range(num_images_per_class_dummy):
#         dummy_img_path = os.path.join(class_train_path, f'{food_class}_{i:03d}.jpg')
#         train_image_paths[food_class].append(dummy_img_path)
#         # Create a dummy blank image (replace with actual image creation if you want visible images)
#         dummy_image = np.zeros((64, 64, 3), dtype=np.uint8)
#         cv2.imwrite(dummy_img_path, dummy_image)

#     for i in range(num_test_images_per_class_dummy):
#         dummy_img_path = os.path.join(class_test_path, f'{food_class}_test_{i:03d}.jpg')
#         test_image_paths[food_class].append(dummy_img_path)
#         dummy_image = np.zeros((64, 64, 3), dtype=np.uint8)
#         cv2.imwrite(dummy_img_path, dummy_image)
# print("Dummy data paths and files created.")
# -----------------------------------------------------------------------------

In [17]:
# --- 2. Data Loading and Preprocessing Function ---

def load_and_preprocess_images(image_paths_dict, target_size=(64, 64)):
    """
    Loads images from paths, resizes them, and prepares them for CNN input.

    Args:
        image_paths_dict (dict): A dictionary where keys are class names and
                                 values are lists of image file paths.
        target_size (tuple): The desired (width, height) for resizing images.

    Returns:
        tuple: A tuple containing:
            - np.array of loaded and preprocessed images (normalized to [0,1]).
            - np.array of corresponding labels (strings).
    """
    images = []
    labels = []
    for food_class, paths in image_paths_dict.items():
        for path in paths:
            img = cv2.imread(path)
            if img is not None:
                # Convert BGR to RGB (OpenCV loads in BGR)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                # Resize image
                img_resized = cv2.resize(img, target_size)
                # Normalize pixel values to [0, 1]
                images.append(img_resized / 255.0)
                labels.append(food_class)
            else:
                print(f"Warning: Could not load image: {path}. Skipping.")
    return np.array(images), np.array(labels)

In [18]:
# --- 3. CNN Model Architecture Design ---

def build_basic_cnn(input_shape, num_classes):
    """
    Builds a basic Convolutional Neural Network model.

    Args:
        input_shape (tuple): Shape of the input images (height, width, channels).
        num_classes (int): The number of distinct food categories.

    Returns:
        tf.keras.Model: The compiled Keras CNN model.
    """
    model = Sequential()

    # First Convolutional Block
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))

    # Second Convolutional Block
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    # Third Convolutional Block (optional, can remove for even simpler model)
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    # Flatten the 3D output to 1D for the Dense layers
    model.add(Flatten())

    # Dense (Fully Connected) Layers
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5)) # Dropout for regularization to prevent overfitting

    # Output layer
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [20]:
# --- 4. Main Execution (Training and Testing) ---

if __name__ == "__main__":
    # Define target image size for CNN input
    IMAGE_SIZE = (128, 128) # Height, Width
    CHANNELS = 3          # Color images (RGB)

    print("--- Starting Data Loading and Preprocessing ---")
    X_train_raw, y_train_labels = load_and_preprocess_images(train_image_paths, target_size=IMAGE_SIZE)
    X_test_raw, y_test_labels = load_and_preprocess_images(test_image_paths, target_size=IMAGE_SIZE)

    if X_train_raw.size == 0 or X_test_raw.size == 0:
        print("Error: No images loaded. Please check your `train_image_paths` and `test_image_paths`.")
        print("Ensure the paths are correct and images exist.")
        exit() # Exit if no data to process

    print(f"Loaded {len(X_train_raw)} training images and {len(X_test_raw)} testing images.")

    # Encode labels (string labels to numerical IDs)
    label_encoder = LabelEncoder()
    all_labels = np.concatenate((y_train_labels, y_test_labels))
    label_encoder.fit(all_labels) # Fit on all unique labels

    y_train_encoded = label_encoder.transform(y_train_labels)
    y_test_encoded = label_encoder.transform(y_test_labels)

    num_classes = len(label_encoder.classes_)
    print(f"Found {num_classes} classes: {label_encoder.classes_}")

    # One-Hot Encode labels (numerical IDs to binary class matrices)
    y_train_one_hot = to_categorical(y_train_encoded, num_classes=num_classes)
    y_test_one_hot = to_categorical(y_test_encoded, num_classes=num_classes)

    print(f"Training data shape: {X_train_raw.shape}")
    print(f"Training labels shape: {y_train_one_hot.shape}")
    print(f"Testing data shape: {X_test_raw.shape}")
    print(f"Testing labels shape: {y_test_one_hot.shape}")

    # --- Build the CNN Model ---
    input_shape = (IMAGE_SIZE[0], IMAGE_SIZE[1], CHANNELS)
    model = build_basic_cnn(input_shape, num_classes)
    print("\n--- Model Summary ---")
    model.summary()


    # --- Train the Model ---
    print("\n--- Starting Model Training ---")
    # It's good practice to use a validation_split or a separate validation set
    # from your training data to monitor for overfitting during training.
    history = model.fit(X_train_raw, y_train_one_hot,
                        epochs=50,       # Number of training iterations
                        batch_size=32,   # Number of samples per gradient update
                        validation_split=0.1, # Use 10% of training data for validation
                        verbose=1)

    # --- Evaluate the Model on the Test Set ---
    print("\n--- Evaluating Model on Test Data ---")
    loss, accuracy = model.evaluate(X_test_raw, y_test_one_hot, verbose=0)
    print(f"Test Loss: {loss:.4f}")
    print(f"Test Accuracy: {accuracy:.4f}")

    # --- Visualize Training History ---
    plt.figure(figsize=(12, 5))

    # Plot training & validation accuracy values
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    # --- Optional: Make and Display Sample Predictions ---
    print("\n--- Making Sample Predictions ---")
    # Select a few random images from the test set for prediction
    num_sample_predictions = min(10, len(X_test_raw))
    random_indices = random.sample(range(len(X_test_raw)), num_sample_predictions)

    sample_images = X_test_raw[random_indices]
    sample_true_labels_encoded = y_test_encoded[random_indices]

    predictions = model.predict(sample_images)
    predicted_classes_encoded = np.argmax(predictions, axis=1)

    print("\n--- Sample Predictions Report ---")
    for i in range(num_sample_predictions):
        true_label = label_encoder.inverse_transform([sample_true_labels_encoded[i]])[0]
        predicted_label = label_encoder.inverse_transform([predicted_classes_encoded[i]])[0]

        # Display the image
        plt.imshow(sample_images[i])
        plt.title(f"True: {true_label}\nPredicted: {predicted_label} (Confidence: {np.max(predictions[i]):.2f})")
        plt.axis('off')
        plt.show()

        print(f"Sample {i+1}: True Label: {true_label}, Predicted Label: {predicted_label}, Confidence: {np.max(predictions[i]):.2f}")


Output hidden; open in https://colab.research.google.com to view.